In [2]:
#Load the dataset
from datasets import load_dataset

import json

# Load the HateXplain dataset
dataset = load_dataset("../hatexplain/hatexplain.py")

# Inspect the dataset
print(dataset)

DatasetDict({
    train: Dataset({
        features: ['id', 'annotators', 'rationales', 'post_tokens'],
        num_rows: 15383
    })
    validation: Dataset({
        features: ['id', 'annotators', 'rationales', 'post_tokens'],
        num_rows: 1922
    })
    test: Dataset({
        features: ['id', 'annotators', 'rationales', 'post_tokens'],
        num_rows: 1924
    })
})


In [3]:
train_data = dataset["train"]
val_data = dataset["validation"]
test_data = dataset["test"]

print(train_data)
print(val_data)
print(test_data)



Dataset({
    features: ['id', 'annotators', 'rationales', 'post_tokens'],
    num_rows: 15383
})
Dataset({
    features: ['id', 'annotators', 'rationales', 'post_tokens'],
    num_rows: 1922
})
Dataset({
    features: ['id', 'annotators', 'rationales', 'post_tokens'],
    num_rows: 1924
})


In [5]:
for i, row in enumerate(val_data):
    post_tokens = row["post_tokens"]
    if "muzzie" in post_tokens and "shithole" in post_tokens:
        print(row)
    

{'id': '15473637_gab', 'annotators': {'label': [0, 0, 0], 'annotator_id': [12, 9, 3], 'target': [['Other'], ['Women', 'Islam'], ['Women', 'Islam']]}, 'rationales': [[0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0], [0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1]], 'post_tokens': ['democrats', 'from', 'minnesota', 'leftist', 'muzzie', 'shithole', 'are', 'the', 'dumbest', 'anti', 'americans', 'there', 'are', 'klobuchar', 'one', 'of', 'the', 'reps', 'is', 'a', 'dumb', 'bitch', 'who', 'has', 'a', 'moronic', 'position', 'on', 'the', 'tax', 'bill', 'fuck', 'this', 'stupid', 'woman']}


In [4]:
for i in range(5):
    print(train_data[i])

{'id': '23107796_gab', 'annotators': {'label': [0, 2, 2], 'annotator_id': [203, 204, 233], 'target': [['Hindu', 'Islam'], ['Hindu', 'Islam'], ['Hindu', 'Islam', 'Other']]}, 'rationales': [[0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]], 'post_tokens': ['u', 'really', 'think', 'i', 'would', 'not', 'have', 'been', 'raped', 'by', 'feral', 'hindu', 'or', 'muslim', 'back', 'in', 'india', 'or', 'bangladesh', 'and', 'a', 'neo', 'nazi', 'would', 'rape', 'me', 'as', 'well', 'just', 'to', 'see', 'me', 'cry']}
{'id': '9995600_gab', 'annotators': {'label': [2, 2, 0], 'annotator_id': [27, 6, 4], 'target': [['Minority', 'Refugee', 'Indian'], ['Indian'], ['Refugee', 'Islam']]}, 'rationales': [[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 

In [6]:
#peak into the sentence
sentence_to_find = "no liberal congratulated hindu refugee post cab because they hate hindus"
for i, row in enumerate(train_data):
    sentence = " ".join(row['post_tokens'])  # Combine tokens into a sentence
    if "hindu" in sentence and "liberal" in sentence and "congratulated" in sentence:
        print(f"Found at index: {i}")
        break
        # print(test_data[i])
        # break
else:
    print("Sentence not found.")

Found at index: 3


In [12]:
count = 0
for i, row in enumerate(val_data):
    labels = row['annotators']['label']
    if len(set(labels))==1:
        count+=1
    if i <10:
        print(row)
print(count)

{'id': '1178613994371928065_twitter', 'annotators': {'label': [1, 1, 1], 'annotator_id': [83, 3, 51], 'target': [['None'], ['None'], ['None']]}, 'rationales': [], 'post_tokens': ['me', 'getting', 'books', 'from', 'the', 'library', 'about', 'queer', 'cowboys', 'and', 'homoeroticism', 'in', 'cinema', 'instead', 'of', 'anything', 'useful', 'for', 'my', 'thesis']}
{'id': '1170285336221638656_twitter', 'annotators': {'label': [1, 1, 1], 'annotator_id': [8, 6, 115], 'target': [['None'], ['None'], ['None']]}, 'rationales': [], 'post_tokens': ['y', 'si', 'fuera', 'top', '<number>', 'me', 'faltarían', 'how', 'to', 'get', 'away', 'with', 'murder', 'gossip', 'girl', 'the', 'last', 'ship', 'orphan', 'black', 'downton', 'abbey']}
{'id': '1179099934731190272_twitter', 'annotators': {'label': [1, 1, 1], 'annotator_id': [17, 1, 18], 'target': [['None'], ['None'], ['None']]}, 'rationales': [], 'post_tokens': ['<user>', '<user>', '<user>', 'i', 'am', 'a', 'lesbian', 'not', 'queer', 'and', 'quite', 'fran

In [3]:
#Focus on train data for now
#Observe the train data

#Count the number of annotators
count_annotators ={}
count_mistmatch = 0
for data in train_data:
    count = len(data['annotators']['annotator_id'])
    if count !=len(data["annotators"]["label"]) or count !=count != len(data["annotators"]["target"]):
        count_mistmatch +=1
    if count in count_annotators:
        count_annotators[count] +=1
    else:
        count_annotators[count] = 1

print(count_annotators)
print("count_mistmatch", count_mistmatch)


{3: 15383}
count_mistmatch 0


In [5]:
#count an item where the number of annotators don't match the number of rationales
count_rationale_match_annotators = 0
count_rational_not_match_annotators = 0
count_rationales = {}
for data in train_data:
    count = len(data['rationales'])
    if count in count_rationales:
        count_rationales[count] += 1
    else:
        count_rationales[count] = 1
        
    if len(data['annotators']["label"]) == count:
        count_rationale_match_annotators += 1
    else:
        count_rational_not_match_annotators += 1
        
print("count_rationale_match_annotators: ", count_rationale_match_annotators)
print("count_rational_not_match_annotators", count_rational_not_match_annotators)
print(count_rationales)

count_rationale_match_annotators:  3518
count_rational_not_match_annotators 11865
{2: 5614, 3: 3518, 0: 6251}


In [15]:
#Are there data labeled as offensive or hate speech but no annotation?
no_rationale_data_list = []
for data in train_data:
    count = len(data["rationales"])
    if count == 0:
        no_rationale_data_list.append(data["annotators"]["label"])

no_rationale_normal = []
no_rationale_else = []
for data in no_rationale_data_list:
    if data[0] == data[1] == data[2] ==1: #normal
        no_rationale_normal.append(data)
    else:
        no_rationale_else.append(data)

print("no_rationale_normal (all three anotators thought the speech was normal)", len(no_rationale_normal))
print("no_rationale_else (not all through it was normal)", len(no_rationale_else))

no_rationale_abnormal = []
for data in no_rationale_else:
    count = 0
    for i in data:
        if i == 1:
            count +=1
    if count < 2:
        no_rationale_abnormal.append(data)
print("no_rationale_abnormal (the majority vote was not normal): ", len(no_rationale_abnormal))


no_rationale_normal (all three anotators thought the speech was normal) 4096
no_rationale_else (not all through it was normal) 2155
no_rationale_abnormal (the majority vote was not normal):  0


In [17]:
#check # of rationales vs # of # of hate speech labels
count_hate_label_match_rationales =0
count_hate_label_no_match_rationales =0
for data in train_data:
    count_hate_label = data["annotators"]["label"].count(0)
    count_rationales = len(data["rationales"])
    if count_hate_label == count_rationales:
        count_hate_label_match_rationales += 1
    else:
        count_hate_label_no_match_rationales += 1
        
print("count_hate_label_match_rationales: ", count_hate_label_match_rationales)
print("count_hate_label_no_match_rationales", count_hate_label_no_match_rationales)
    

count_hatel_label_match_rationales:  9016
count_hatel_label_no_match_rationales 6367


In [13]:
#Check target
target_if_normal = []
for data in train_data:
    labels = data["annotators"]["label"]
    if labels[0]==labels[1]==labels[2] == 1: #normal
        target_if_normal.append(data["annotators"]["target"])

print(target_if_normal[:100])

[[['Refugee'], ['Refugee'], ['None']], [['Arab', 'Caucasian'], ['None'], ['Arab']], [['Women'], ['None'], ['Women']], [['None'], ['None'], ['None']], [['None'], ['None'], ['None']], [['Islam', 'Women'], ['Arab', 'Islam', 'Women'], ['Arab', 'Islam']], [['None'], ['African'], ['None']], [['None'], ['None'], ['None']], [['None'], ['None'], ['None']], [['None'], ['None'], ['None']], [['None'], ['None'], ['None']], [['Other'], ['None'], ['Other']], [['None'], ['Women', 'Homosexual'], ['None']], [['None'], ['Other'], ['None']], [['None'], ['None'], ['None']], [['None'], ['None'], ['None']], [['None'], ['None'], ['None']], [['None'], ['None'], ['None']], [['None'], ['None'], ['None']], [['None'], ['Other'], ['None']], [['None'], ['None'], ['None']], [['None'], ['Women', 'None'], ['Other']], [['None'], ['None'], ['None']], [['None'], ['None'], ['None']], [['None'], ['Islam'], ['None']], [['None'], ['None'], ['None']], [['None'], ['None'], ['None']], [['None'], ['Caucasian'], ['None']], [['None

In [17]:
#check annotations
from collections import Counter
annotation_count = {"absent": {"hate": 0, "normal": 0, "offensive":0}, "present": {2: {"hate": 0, "normal": 0, "offensive":0}, 3: {"hate": 0, "offensive":0}}, "match": {2: {"hate": 0,  "offensive":0}, 3: {"hate": 0, "offensive":0}}, "no_match": {2: {"hate": 0, "offensive":0}, 3: {"hate": 0, "offensive":0}}}
no_match_examples = []
def get_majority_element(lst):
        # Count occurrences of each element
        count = Counter(lst)
        
        # Check if there is a majority (i.e., an element that appears at least twice)
        if count.most_common(1)[0][1] >= 2:
            return count.most_common(1)[0][0]
        else:
            return 3  # Indicate undecided if no majority
        
for data in train_data:
    annotations = data["rationales"]
    labels = data["annotators"]['label']
    label_token = get_majority_element(labels)
    label_mapping={0: "hate", 1: "normal", 2: "offensive", 3: "undecided"}
    label = label_mapping[label_token]
    if len(annotations)==0:
        annotation_count["absent"][label]+=1
    else:
        annotation_count["present"][len(annotations)][label]+=1
        if len(annotations)==2:
            if annotations[0]==annotations[1]:
                annotation_count["match"][len(annotations)][label]+=1
            else:
                annotation_count["no_match"][len(annotations)][label]+=1
                no_match_examples.append(data)
        elif len(annotations)==3:
            if annotations[0]==annotations[1] and annotations[1]==annotations[2]:
                annotation_count["match"][len(annotations)][label]+=1
            else:
                annotation_count["no_match"][len(annotations)][label]+=1
print(annotation_count)
print(no_match_examples[:5])
        

{'absent': {'hate': 0, 'normal': 6251, 'offensive': 0}, 'present': {2: {'hate': 2049, 'normal': 0, 'offensive': 3565}, 3: {'hate': 2699, 'offensive': 819}}, 'match': {2: {'hate': 383, 'offensive': 866}, 3: {'hate': 693, 'offensive': 282}}, 'no_match': {2: {'hate': 1666, 'offensive': 2699}, 3: {'hate': 2006, 'offensive': 537}}}
[{'id': '23107796_gab', 'annotators': {'label': [0, 2, 2], 'annotator_id': [203, 204, 233], 'target': [['Hindu', 'Islam'], ['Hindu', 'Islam'], ['Hindu', 'Islam', 'Other']]}, 'rationales': [[0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]], 'post_tokens': ['u', 'really', 'think', 'i', 'would', 'not', 'have', 'been', 'raped', 'by', 'feral', 'hindu', 'or', 'muslim', 'back', 'in', 'india', 'or', 'bangladesh', 'and', 'a', 'neo', 'nazi', 'would', 'rape', 'me', 'as', 'well', 'just', 'to', 'see', 'me', 'cry']}, {'id': '12

In [33]:
#visualize annotations
def visualize(data):
    post_tokens = data["post_tokens"]
    rationales = data["rationales"]
    
    sentence = " ".join(post_tokens)  # Joins words with a space in between
    print(sentence)

    for rationale in rationales:
        print("rationale: ")
        for i in range(len(rationale)):
            if rationale[i]==1:
                print(post_tokens[i])
        print()
for j in range(5):
    visualize(no_match_examples[j])

u really think i would not have been raped by feral hindu or muslim back in india or bangladesh and a neo nazi would rape me as well just to see me cry
rationale: 
raped
neo
nazi
would
rape

rationale: 
raped
feral
hindu
muslim
neo
nazi

if english is not imposition then hindi is also not imposition shut up chutiya retards stophindiimposition
rationale: 
chutiya
retards
stophindiimposition

rationale: 
retards

<user> <number> americans dead because of an illegal immigrant this is infuriating enforce our laws deport them
rationale: 
<number>
americans
dead
because
of
an
illegal
immigrant

rationale: 
deport
them

thank a white man you live in his world and why do you islanders say you are canadian so rude really if i lived in a brown country even with citizenship i would never call myself ethiopian chinese indian you are a caribbean living in canada
rationale: 
brown
country
you
are
a
caribbean
living
in
canada

rationale: 
and
why
do
you
islanders
say
you
are
canadian
so
rude
really
i

In [41]:
rationale_length = {"match": 0, "no_match": 0}
no_length_match_samples = []

for data in train_data:
    rationales = data['rationales']
    if len(rationales)==0:
        continue
    sentence_length_1 = len(rationales[0])
    sentence_length_2 = len(rationales[1])
    if sentence_length_1 !=sentence_length_2:
        rationale_length["no_match"] +=1
        no_length_match_samples.append(data)
    if len(rationales)==2:
        if sentence_length_1 ==sentence_length_2:
            rationale_length["match"] +=1
        continue
    if sentence_length_2 !=len(rationales[2]):
        rationale_length["no_match"] +=1
        no_length_match_samples.append(data)
    else:
        rationale_length["match"] +=1
print(rationale_length)
print(no_length_match_samples)
#There is one sample where length does not match

print(len(no_length_match_samples[0]["post_tokens"]))
print(len(no_length_match_samples[0]["rationales"][0]))
print(len(no_length_match_samples[0]["rationales"][1])) #Cut the last word since it's one word off.

{'match': 9131, 'no_match': 1}
[{'id': '24439295_gab', 'annotators': {'label': [2, 2, 0], 'annotator_id': [222, 209, 201], 'target': [['Homosexual'], ['Other'], ['Homosexual']]}, 'rationales': [[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 1, 1, 1]], 'post_tokens': ['my', 'rhymes', 'pass', 'any', 'bar', 'exam', 'they', 'call', 'me', 'kincannon', '<number>', 'shots', 'to', 'your', 'think', 'tank', 'im', 'wavin', 'twin', 'cannons', 'you', 'a', 'fag', 'that', 'loves', 'it', 'up', 'the', 'ass', 'you', 'are', 'a', 'ken', 'kaniff', 'and', 'yo', 'bitch', 'gimme', 'sloppy', 'toppy', 'they', 'call', 'her', 'steve', 'bannon', 'the', 'g.o.a.t.']}]
47
47
48
my rhymes pass any bar exam they call me kincannon <number> shots to your think tank im wavin twin cannons you a 

IndexError: list index out of range

In [14]:
for data in train_data: 
    if data["id"] == "1190398852601384960_twitter":
        print(data)

{'id': '1190398852601384960_twitter', 'annotators': {'label': [1, 2, 2], 'annotator_id': [236, 212, 211], 'target': [['Asian'], ['Asian'], ['Asian']]}, 'rationales': [[0, 0, 0, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0], [0, 0, 0, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0]], 'post_tokens': ['bro', 'why', 'do', 'asians', 'have', 'a', 'low', 'alcohol', 'tolerance', 'this', 'is', 'not', 'fair']}


In [6]:
for data in train_data:
    post_tokens = data["post_tokens"]
    if post_tokens == ['muslims', 'have', 'been', 'enslaving', 'and', 'raping', 'white', 'women', 'for', '<number>', 'years', 'european', 'painters', 'were', 'calling', 'attention', 'to', 'this', 'for', 'centuries', 'today', 'no', 'one', 'cares', 'enjoy', 'being', 'raped', 'european', 'white', 'women']:
        print(data["id"])
        keywords = ["muslims", "enslaving", "raping", "enjoy", "being", "raped" ]
        rationale = [0 for post in range( len(post_tokens))]
        for i in range(len(post_tokens)):
            if post_tokens[i] in keywords:
                rationale[i] = 1.0
        print (rationale)
    if post_tokens == ['<user>', 'not', 'good', 'good', 'vs', 'bad', 'more', 'like', 'they', 'both', 'suck', 'choosing', 'sides', 'is', 'what', 'the', 'zionist', 'and', 'the', 'arabs', 'like', 'i', 'could', 'care', 'less', 'if', 'they', 'kill', 'each', 'other', 'nobloodforisrael', 'we', 'have', 'our', 'own', 'problems', 'our', 'gov', 'is', 'commitiing', 'high', 'treason', 'while', 'being', 'tyrannical']:
        print(data["id"])
        keywords = ["zionist", "arabs", "kill", "each", "other", "nobloodforisrael" ]
        rationale = [0 for post in range( len(post_tokens))]
        for i in range(len(post_tokens)):
            if post_tokens[i] in keywords:
                rationale[i] = 1.0
        print (rationale)
                

4008588_gab
[1.0, 0, 0, 1.0, 0, 1.0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1.0, 1.0, 1.0, 0, 0, 0]
4004699_gab
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1.0, 0, 0, 1.0, 0, 0, 0, 0, 0, 0, 0, 1.0, 1.0, 1.0, 1.0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]


In [3]:
#Let's process the training/eval data
from collections import Counter

def compute_average_attention(rationales):
    """
    Computes the average attention value for each token across a list of rationales.

    Args:
        rationales (list of lists): Each rationale is a list representing token-level attention (0 or 1).

    Returns:
        list: Average attention vector for each token.
    """
    # Number of rationales
    num_rationales = len(rationales)

    # Sentence length (assuming all rationales are the same length)
    sentence_length = len(rationales[0])

    # Initialize a list to store the sums of token-wise values
    token_sums = [0] * sentence_length

    # Sum up the token-wise values for each rationale
    for rationale in rationales:
        for i, value in enumerate(rationale):
            token_sums[i] += value

    # Compute the average by dividing the sums by the number of rationales
    average_attention = [token_sum / num_rationales for token_sum in token_sums]

    return average_attention

def process_easy_data(dataset):
    def get_majority_element(lst):
        # Count occurrences of each element
        count = Counter(lst)
        
        # Check if there is a majority (i.e., an element that appears at least twice)
        if count.most_common(1)[0][1] >= 2:
            return count.most_common(1)[0][0]
        else:
            return 3  # Indicate undecided if no majority
        
    easy_data =[]
    for data in dataset:
        labels = data["annotators"]['label']
        targets = data["annotators"]["target"]
        rationales = data["rationales"]
        id = data["id"]
        
        final_label = get_majority_element(labels)
        
        final_targets = []
        target_dicts = {}
        for target in targets:
            for word in target:
                if word in target_dicts:
                    target_dicts[word] += 1
                else:
                    target_dicts[word] = 1
        for dict in target_dicts:
            if target_dicts[dict] >=2:
                final_targets.append(dict)
                
        #fix the bug in the dataset
        if id =="24439295_gab":
            rationales[1].pop() #fix the length mismatch
            
        if len(rationales)==0:
            rationale = []
        else:
            #fix the bug
            if id == "4008588_gab":
                rationale = [1.0, 0, 0, 1.0, 0, 1.0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1.0, 1.0, 1.0, 0, 0, 0]
            elif id == "4004699_gab":
                rationale = [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1.0, 0, 0, 1.0, 0, 0, 0, 0, 0, 0, 0, 1.0, 1.0, 1.0, 1.0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]           
            else:
                rationale = compute_average_attention(rationales)
                                              
        final_data = {}
        final_data["id"] = id 
        final_data["post_tokens"] = data["post_tokens"]
        final_data["label"] = final_label
        final_data["target"] = final_targets
        final_data["rationale"] = rationale
        easy_data.append(final_data)
    return easy_data

In [11]:
#Processing train data
import numpy as np
easy_train_data = process_easy_data(train_data)
print("easy_train_data length: ", len(easy_train_data))
print(easy_train_data[:10])
# Save list as a JSON file
with open('easy_train_data.json', 'w') as f:
    json.dump(easy_train_data, f)
    


easy_train_data length:  15383
[{'id': '23107796_gab', 'post_tokens': ['u', 'really', 'think', 'i', 'would', 'not', 'have', 'been', 'raped', 'by', 'feral', 'hindu', 'or', 'muslim', 'back', 'in', 'india', 'or', 'bangladesh', 'and', 'a', 'neo', 'nazi', 'would', 'rape', 'me', 'as', 'well', 'just', 'to', 'see', 'me', 'cry'], 'label': 2, 'target': ['Hindu', 'Islam'], 'rationale': [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.5, 0.5, 0.0, 0.5, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.5, 0.5, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]}, {'id': '9995600_gab', 'post_tokens': ['the', 'uk', 'has', 'threatened', 'to', 'return', 'radioactive', 'waste', 'to', 'the', 'eu', 'if', 'an', 'agreement', 'cannot', 'be', 'reached', 'can', 'not', 'we', 'keep', 'the', 'radioactive', 'waste', 'and', 'send', 'back', 'all', 'the', 'paki', 'migrants', 'instead'], 'label': 2, 'target': ['Refugee', 'Indian'], 'rationale': [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,

In [15]:
#Process eval data
easy_eval_data = process_easy_data(val_data)
print("easy_eval_data length: ", len(easy_eval_data))
print(easy_eval_data[:10])

# Save list as a JSON file
with open('easy_eval_data.json', 'w') as f:
    json.dump(easy_eval_data, f)

easy_eval_data length:  1922
[{'id': '1178613994371928065_twitter', 'post_tokens': ['me', 'getting', 'books', 'from', 'the', 'library', 'about', 'queer', 'cowboys', 'and', 'homoeroticism', 'in', 'cinema', 'instead', 'of', 'anything', 'useful', 'for', 'my', 'thesis'], 'label': 1, 'target': ['None'], 'rationale': []}, {'id': '1170285336221638656_twitter', 'post_tokens': ['y', 'si', 'fuera', 'top', '<number>', 'me', 'faltarían', 'how', 'to', 'get', 'away', 'with', 'murder', 'gossip', 'girl', 'the', 'last', 'ship', 'orphan', 'black', 'downton', 'abbey'], 'label': 1, 'target': ['None'], 'rationale': []}, {'id': '1179099934731190272_twitter', 'post_tokens': ['<user>', '<user>', '<user>', 'i', 'am', 'a', 'lesbian', 'not', 'queer', 'and', 'quite', 'frankly', 'his', 'disco', 'tits', 'video', 'was', 'repulsive', 'if', 'he', 'thinks', 'being', 'an', 'overweight', 'gay', 'bloke', 'brings', 'out', 'the', 'woman', 'in', 'him', 'then', 'i', 'am', 'offended', 'it', 'pathetic', 'attention', 'seeking'],

In [4]:
#Processing train data
import numpy as np
easy_test_data = process_easy_data(test_data)
print("easy_test_data length: ", len(easy_test_data))
# Save list as a JSON file
with open('easy_test_data.json', 'w') as f:
    json.dump(easy_test_data, f)
    

easy_test_data length:  1924


In [6]:
import json

# Load the dataset from a JSON file
with open("easy_train_data.json", "r") as f:
    easy_train_data = json.load(f)

# Print the first element in the loaded dataset
label_counts ={}
for data in easy_train_data:
    label = data["label"]
    if label in label_counts:
        label_counts[label] +=1
    else:
        label_counts[label] =1
print(label_counts)


{2: 4384, 1: 6251, 0: 4748}


In [7]:
import json

# Load the dataset from a JSON file
with open("easy_eval_data.json", "r") as f:
    easy_eval_data = json.load(f)

label_counts ={}
for data in easy_eval_data:
    label = data["label"]
    if label in label_counts:
        label_counts[label] +=1
    else:
        label_counts[label] =1
print(label_counts)

{1: 781, 0: 593, 2: 548}


In [8]:
import json

# Load the dataset from a JSON file
with open("easy_test_data.json", "r") as f:
    easy_eval_data = json.load(f)

label_counts ={}
for data in easy_test_data:
    label = data["label"]
    if label in label_counts:
        label_counts[label] +=1
    else:
        label_counts[label] =1
print(label_counts)

{1: 782, 2: 548, 0: 594}


In [17]:
target_counts ={}
for data in easy_eval_data:
    targets = data['target']
    # print(targets)
    for target in targets:
        target = target.lower()
        if target in target_counts:
            target_counts[target] += 1
        else:
            target_counts[target]=12
print(target_counts)

{'none': 668, 'women': 151, 'african': 310, 'refugee': 77, 'islam': 204, 'homosexual': 182, 'jewish': 186, 'disability': 4, 'caucasian': 48, 'other': 84, 'hispanic': 35, 'arab': 77, 'asian': 34, 'christian': 3, 'men': 6}
